# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [301]:
text = open('/Users/victoriaregina/Documents/assignment1/text.txt', 'r').read()[2:]

In [302]:
import string
import re

def preprocess_text(text):
  # TODO
  # make lowercase
    text = text.lower()

  # replace all punctuation except dots with spaces
    text = "".join(l for l in text if l not in '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~»«—…‘’“”')
  
  # collapse multiple spaces into one '   ' -> ' '
    text = ' '.join(text.split())
 
    return text


text2 = preprocess_text(text)
itos = list(set(text2))


In [303]:
text3 = text2.split('.')
text3 = [x.strip() for x in text3]
len(text3)

30876

In [304]:
from collections import Counter
import nltk
from sklearn.base import TransformerMixin


class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        syms = set()
      
      # tokenize text by symbols and fill in self.itos and self.stoi
        for phrase in text:
            for symb in phrase:
                syms.add(symb)
          
        self.itos = list(syms)

        for n_sym, sym in enumerate(self.itos):
            self.stoi[sym] = n_sym
      
        coded_text = []
        for string in text:
            
            coded_string = []
            for s in string:
                coded_string.append(self.stoi[s])
            
            coded_text.append(coded_string)
   
        text4 = coded_text
        
        while len(self.itos) < self.vocab_size:
            #print(len(self.itos))
            pairs = {}

      # count bigram freqencies in the text

            for sentence in text4:
              for i, t in enumerate(sentence):
                if i < len(sentence)-1:
                    p = (sentence[i], sentence[i+1])
                    if p not in pairs:
                        pairs[p] = 1
                    else:
                        pairs[p] += 1
          
          # most common bigram in the text
        
            most_freq_bigram = sorted(pairs.items(), key=lambda x: (x[1],x[0]), reverse=True)[0][0]

            
          #convert most frequent bigram into new_token to find it in the text
            new_token = ''
            for e in most_freq_bigram:
                #print(e)
                new_token = new_token + ' ' + str(e)
                #print(new_token)
            
            self.itos.append(new_token)
            self.stoi[new_token] = len(self.itos)

          #find occurences of the new_token in the text and replace them with new_id
            text5 = []
            for string in text4:
                stroka_arr = []
                for s in string:
                    stroka_arr.append(str(s))
                    stroka = ' '.join(stroka_arr)
                #print(stroka)
                
            #print('changed string', stroka)
            
                #stroka = stroka.replace(new_token, ' ' + str(self.stoi[most_freq_bigram]))
                stroka = stroka.replace(new_token, ' ' + str(self.stoi[new_token]))
                temp_st = []
                for s in stroka.split():
                    temp_st.append(int(s))
                text5.append(temp_st)
            #print('temp', text5)

            text4 = text5
          
        return self
    
    def transform(self, text):
        
        #tokenize text by symbols using self.stoi
        text_transformed = []
    
        for string in text:
       
        #начальный элемент массива Рез
            arRes = []
            arStr = list(string)
            if len(arStr) > 1:
                #print(len(arStr))
        
                if (self.stoi[arStr[0]], self.stoi[arStr[1]]) in self.stoi:
                    p = (self.stoi[arStr[0]], self.stoi[arStr[1]])
                    arRes.append(p)

                else:
                    arRes.append(arStr[0])

                arStr = list(string)
            # сравниваем пару: последний элемент рез-а и текущий элемент строки
            # если есть такая пара вместо последнего Рез-а пишем значение пары, иначе 
            # добавляем новый элемент в Рез
                i = 1                   
                while i <= len(arStr)-1:
                    lastIndex = len(arRes)-1
                    lastEl = arRes[lastIndex] 
                    s = arStr[i]

                    if (self.stoi[lastEl], self.stoi[s]) in self.stoi:
                        p = (self.stoi[lastEl], self.stoi[s])
                        arRes[lastIndex] = p

                    else:
                        arRes.append(s)
                    i += 1

            #перевод резов в цифру

          #find occurences of the token in the text and replace them with token_id

                for num, token in enumerate(self.itos):
                    if token in arRes:
                        for i, el in enumerate(arRes):
                            if el == token:
                                arRes[i] = num
                text_transformed.append(arRes)

        return text_transformed
    
    
    def decode_token(self, tok): 
        """
        tok: int or tuple
        """
        result = self.itos[tok] # TODO
        return result 
            
    def decode(self, text):
        """
        convert token ids into text
        
        """
        
        return ''.join(map(self.decode_token, text))
    

In [ ]:
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text3)

In [145]:
assert bpe.decode(tokenized_text[0]) == text3[0]

In [274]:
import numpy as np
  
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        # create array for storing 3-gram counters
        self.proba = Counter()
        
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        
        freqs = []
        freq_sum = 0
        for word in range(self.vocab_size):
            word_counter = self.get_proba(a, b, word, tau=tau)
            freqs.append(word_counter)
            freq_sum += word_counter
        result = [f/freq_sum for f in freqs]
        
        return np.array(result)
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        # approximate probability by counters
        
        trigram =  self.proba[(a, b, c)]
        result = (trigram + self.delta) ** (1/tau) 
        
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        # count 3-grams in the text
        
        for t in text:
            #for e in t:
            t = [start_token] + [t_el for t_el in t] + [end_token]
                #t = [start_token] + [e] + [end_token]
                
            self.proba.update((zip(t, t[1:], t[2:])))
            
        return self
    

In [275]:
lm = LM(vocab_size, 1).fit(tokenized_text)

In [284]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    beam = [(input_seq, 0)] # TODO store in beam tuples of current sequences and their log probabilities
    
    for i in range(max_len):
        candidates = []
        candidates_proba = []
        for snt, snt_proba in beam:
            if np.argmax(snt[-1]) == end_token:
                continue
            else:
                proba = lm.infer(snt[-2], snt[-1], tau) # probability vector of the next token
                
                best_k = proba.argsort()[-k:][::-1] # top-k most probable tokens
                
                # TODO update candidates' sequences and corresponding probabilities
                    
                candidates += [snt + [int(best)] for best in best_k]
                candidates_proba += [snt_proba + best for best in np.log(proba[best_k])]

        # select top-k most probable sequences from candidates
        beam = [(candidates[i], candidates_proba[i]) for i in np.argsort(candidates_proba).astype(int)[-k:][::-1]]
                
    return beam
    

In [291]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for (snt, snt_proba) in result:
    print('probability of the sequence:', bpe.decode(snt), 'is', np.exp(snt_proba))

probability of the sequence: horse and the an is 0.44203236747819247
probability of the sequence: horse the and th is 0.13678898877003134
probability of the sequence: horse and the th is 0.13678898877003132
probability of the sequence: horse the the an is 0.028786621492962825
probability of the sequence: horse and the sa is 0.018796482870682373
probability of the sequence: horse and the so is 0.015418175164731457
probability of the sequence: horse said the a is 0.014567315524745517
probability of the sequence: horse and the wh is 0.012442495231459908
probability of the sequence: horse was and th is 0.010980752280501927
probability of the sequence: horse and the sh is 0.00921311886762556


In [292]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for (snt, snt_proba) in result:
    print('probability of the sequence:', bpe.decode(snt), 'is', np.exp(snt_proba))

probability of the sequence: her the and t is 0.46393689605804317
probability of the sequence: her and the a is 0.19057696585337455
probability of the sequence: her the the a is 0.09763576857881755
probability of the sequence: her and the t is 0.05897351279437877
probability of the sequence: her the the t is 0.030213117423130327
probability of the sequence: her and the s is 0.020221051706612227
probability of the sequence: her the said  is 0.015520067698379773
probability of the sequence: her the the s is 0.010359583153223632
probability of the sequence: her and the w is 0.009842230386299578
probability of the sequence: her the so th is 0.00937638466403029


In [293]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
for (snt, snt_proba) in result:
    print('probability of the sequence:', bpe.decode(snt), 'is', np.exp(snt_proba))

probability of the sequence: what of the th is 0.00010492944469129061
probability of the sequence: what the and t is 9.515907418415685e-05
probability of the sequence: what the the a is 7.299435842341057e-05
probability of the sequence: what of the an is 7.269115019068714e-05
probability of the sequence: what the the t is 6.491543245869236e-05
probability of the sequence: what the ther  is 6.356741232950716e-05
probability of the sequence: what the and a is 6.179151476380372e-05
probability of the sequence: what and the a is 6.068306031511275e-05
probability of the sequence: what the the s is 5.8326058468719196e-05
probability of the sequence: what the the w is 5.42739721645402e-05


In [294]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for (snt, snt_proba) in result:
    print('probability of the sequence:', bpe.decode(snt), 'is', np.exp(snt_proba))

probability of the sequence: gun the and th is 0.6438173769600412
probability of the sequence: gun the the an is 0.13548844324230633
probability of the sequence: gun the the th is 0.041927534055648544
probability of the sequence: gun the said t is 0.021216691990735045
probability of the sequence: gun and the an is 0.01580860422180541
probability of the sequence: gun the so the is 0.013012211812235578
probability of the sequence: gun the was an is 0.010876350834173001
probability of the sequence: gun the she an is 0.009120817512504905
probability of the sequence: gun the and an is 0.008581012845634669
probability of the sequence: gun the the sa is 0.005761356837076132


In [299]:
from math import exp
from math import log

def perplexity(snt, lm):
    
    """
    snt: sequence of token ids
    lm: language model
    """
    
    sigma = 0
    
    snt = [start_token] + snt + [end_token]
    #print(snt)

    for sym in range(len(snt) - 2):
        sigma += log((1 / lm.infer(snt[sym], snt[sym + 1])[snt[sym + 2]]))
    factor = (-1 / float(len(snt))) * sigma
    result = 2 ** factor
    return result 

In [300]:
perplexity(tokenized_text[0], lm)

0.23800720528860603